In [9]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import math as m

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import os

In [6]:

lb = 1e-8
n_epoch = 2500
n_plot = 10
grad_max = 1e2
maxiters = 50000

lr_max = 5e-3
lr_min = 1e-5
lr_decay = 0.2
lr_decay_step = 500
w_decay = 1e-8

llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3

l_exp = np.arange(1,15)
n_exp = len(l_exp)

l_train = []
l_val = []
for i in np.arange(0,n_exp):
    j = l_exp[i]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i+1)
    else:
        l_val.append(i+1)

# 初始参数设定

In [11]:

lb = 1e-8
n_epoch = 2500
n_plot = 10
grad_max = 1e2
maxiters = 50000

lr_max = 5e-3
lr_min = 1e-5
lr_decay = 0.2
lr_decay_step = 500
w_decay = 1e-8

llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3

# 共十四个实验，依次编号为1，2，...，14
l_exp = np.arange(1,15)
n_exp = len(l_exp)

# 训练集与验证集的数字编号列表，其中[2,6,9,12]为测试集
l_train = []
l_val = []
for i in np.arange(1,n_exp+1):
    j = l_exp[i]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i+1)
    else:
        l_val.append(i+1)

nr=8
ns=4

npara = nr * (ns + 4) + 1

# 实验数据读取

## 目录定义

In [12]:
expr_name = "4s8r-01"
os.chdir("..")
print("当前工作目录：%s"%(str(os.getcwd())))

当前工作目录：w:\Weekly Reportex\No.76


In [ ]:
def load_exp(filename):
    exp_data = np.loadtxt(filename)
    
    # 归一化质量数据
    exp_data[:, 2] = exp_data[:, 2] / max(exp_data[:,2])
    return tf.constant(exp_data, dtype=tf.float64)

l_exp_data = []
l_exp_info = tf.zeros(shape=(len(l_exp), 3))

# 初始化p矩阵

In [4]:
p = tf.Variable(tf.zeros(shape=(npara)))

# 定义CRNN方程

In [ ]:
def p2vec(p):
    w_in = ...
    w_b = ...
    w_out = ...
    return w_in, w_b, w_out

@tf.function
def crnn(t, u):
    du_dt = ...
    return du_dt

# 定义损失函数

In [ ]:
def loss_n_ode(p, i_exp):
    return loss

# 训练过程

In [ ]:
opt = ...
for epoch in range(n_epoch):
    print("Start epoch %d" %(epoch))
    
    for i_exp in (shuffled index of experiments):
        if i_exp in l_val:
            continue

        with tf.GradientTape() as tape:
            tape.watch(p)
            loss = loss_n_ode(p, i_exp)
        
        grad = tape.gradient(loss, p)
        opt.apply_gradients(zip([grad],[p]))
    
